In [1]:
from ACNet_v1 import ACNet
from ACNet_v1 import ReplayMemory
import FireFightingEnv as FFEnv
import torch
import torch.optim as optim
import random
import math
import time
from collections import namedtuple, deque
from itertools import count
import torch.nn as nn

AGT_COUNT = 3
GRID_SIZE = 11

def test_model(policy_net, env, num_episodes=10):
    policy_net.eval()  # 将模型设为评估模式

    for i_episode in range(num_episodes):
        state = env.reset()
        total_reward = 0

        while True:
            # 对每个agent进行推断
            actions = []
            for s in state:
                input = torch.from_numpy(s[0]).float().to(device)  # 将state转换为tensor
                water_res = torch.tensor([[s[1]]]).float().to(device)
                action_i = policy_net.select_action(input, water_res)
                action_int = action_i.item()
                action = [action_int % 5, int(action_int / 5)]
                actions.append(action)

            # 执行动作
            next_state, reward, done, _ = env.step(actions)

            # 更新总奖励
            total_reward += sum(reward)

            # 更新当前状态
            state = next_state

            # 如果episode结束，则打印总奖励并退出循环
            if done:
                print(f"Episode {i_episode + 1}, Total Reward: {total_reward}")
                break

# 创建环境
env = FFEnv.FFEnv(3)  # 请用你实际使用的环境类来替代

# 创建模型实例
policy_net = ACNet(10, GRID_SIZE)  # 请用你实际使用的神经网络类来替代

# 加载模型参数
checkpoint = torch.load('./results/checkpoint/ckpt_336.pth')  # 请用你实际的模型文件路径来替代
policy_net.load_state_dict(checkpoint['network'])

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_net.to(device)

# 测试模型
test_model(policy_net, env, num_episodes=10)

TypeError: __init__() missing 2 required positional arguments: 'n_actions' and 'GRID_SIZE'